In [1]:
import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from dataloader import DataGenerator
from model import attention_unet_refined
# from google.colab.patches import cv2_imshow
import time

import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [3]:
tf.random.set_seed(100)
np.random.seed(100)

## Path
model_path = '/content/drive/My Drive/inPaintingweights.h5' # Saving location
log_path = '/content/drive/My Drive/inPaintingweights.csv'
# weights_paths = ['/content/drive/My Drive/weights_seg_C-V-E-B_train_0.65_val_0.60.h5',
#                  '/content/drive/My Drive/weights_train_C-V-E-B-IR_focalloss_0.02_bifmse_1.0.h5',  # Loading location
#                  '/content/drive/My Drive/weights.h5']  # Loading location

weights_paths = None
train_path = "/content/drive/My Drive/Data/corrected/unlabelled"

## Parameters
image_size = (640, 512) # Original = (2448, 1920)
batch_size = 4
lr = 1e-4
epochs = 300

In [4]:
def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1))

def SSIM(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1))

In [5]:
model = attention_unet_refined(input_shape=image_size, mask_channels=3, out_channels=3, 
                                  multiplier=5, freeze_segmentor=False, 
                                  use_constraints = False, dropout_rate=0.0)
optimizer = Adam(learning_rate=lr)
model.compile(loss= SSIMLoss, optimizer=optimizer, metrics='mse')
model.summary()

# Resume from checkpoint
if weights_paths != None:
    for wp in weights_paths:
        model.load_weights(wp, by_name=True)
        print("Loaded!")

Model: "attention_unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 640, 512, 3) 0                                            
__________________________________________________________________________________________________
conv_1_c1 (Conv2D)              (None, 640, 512, 40) 1120        input[0][0]                      
__________________________________________________________________________________________________
conv_1_d1 (Dropout)             (None, 640, 512, 40) 0           conv_1_c1[0][0]                  
__________________________________________________________________________________________________
conv_1_bn1 (BatchNormalization) (None, 640, 512, 40) 160         conv_1_d1[0][0]                  
_____________________________________________________________________________________

In [ ]:
# Data generators
train_generator = DataGenerator(train_path, batch_size=batch_size, augment=True)

callbacks = [ModelCheckpoint(model_path, save_weights_only=True),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, min_lr=1e-6),
            CSVLogger(log_path),
            # TensorBoard(),
            EarlyStopping(monitor='loss', patience=15, restore_best_weights=False)]

model.fit(x = train_generator, steps_per_epoch = train_generator.__len__(), epochs = epochs, verbose = 1, callbacks = callbacks, workers = 1)

Initialising data generator
Image count in /content/drive/My Drive/Data/corrected/unlabelled path: 1200
Epoch 1/300
128/300 [===========>..................] - ETA: 11:20 - loss: 0.6751 - SSIM: 0.3249